In [4]:
import pymysql
import pandas as pd

In [5]:
df = pd.read_csv('Youtube_data.csv', index_col = 0)
df.head()

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,n8bLutlAfUc,SIDEMEN FAMILY FEUD 2,2022-02-06,6892324.0,386544.0,10559
1,xP8y54cbyk0,"SIDEMEN THAT FINDS $250,000 LAMBORGHINI WINS IT",2022-01-30,6352325.0,385047.0,13557
2,p1dfV9up_MY,SIDEMEN DRINK ONE COLOUR FOR 24 HOURS CHALLENGE,2022-01-23,8084022.0,328922.0,8370
3,Q9hy5ZxXaZA,"SIDEMEN HIDE AND SEEK IN $20,000,000 SPANISH M...",2022-01-16,6279800.0,323609.0,8247
4,PeCBE33DYYI,THE GREATEST SIDEMEN MOMENTS 2021,2022-01-09,4907846.0,308143.0,8587


In [17]:
def connect_to_db(host_name, username, password, dbname):
    try:
        conn = pymysql.connect(host=host_name, user= username, password=password, database=dbname, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
    except pymysql.OperationalError as e:
        raise e
    else:
        print("Connected to database")
    return conn

def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                    video_id VARCHAR(255) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    upload_date DATE NOT NULL,
                    view_count INTEGER NOT NULL,
                    like_count INTEGER NOT NULL,
                    comment_count INTEGER NOT NULL
            )""")

    curr.execute(create_table_command)

def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date,
                        view_count, like_count,comment_count)
    VALUES(%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count, comment_count)
    curr.execute(insert_into_videos, row_to_insert)


def update_row(curr, video_id, video_title, view_count, like_count, comment_count):
    query = ("""UPDATE videos
            SET video_title = %s,
                view_count = %s,
                like_count = %s,
                dislike_count = %s,
                comment_count = %s
            WHERE video_id = %s;""")
    vars_to_update = (video_title, view_count, like_count, comment_count, video_id)
    curr.execute(query, vars_to_update)


def check_if_video_exists(curr, video_id): 
    query = ("""SELECT video_id FROM videos WHERE video_id = %s""")

    curr.execute(query, (video_id,))
    return curr.fetchone() is not None



def truncate_table(curr):
    truncate_table = ("""TRUNCATE TABLE videos""")

    curr.execute(truncate_table)


def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count']
                          , row['like_count'], row['comment_count'])


def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count',
                                   'like_count', 'comment_count'])
    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']): # If video already exists then we will update
            update_row(curr,row['video_id'],row['video_title'],row['view_count'],row['like_count'],row['comment_count'])
        else: # The video doesn't exists so we will add it to a temp df and append it using append_from_df_to_db
            tmp_df = tmp_df.append(row)

    return tmp_df

def delete_table(curr):
    query = ("""DROP TABLE Videos""")
    curr.execute(query)

In [6]:
host_name = 'databaseyt.xxxx.us-east-1.rds.amazonaws.com'
dbname = 'xxxx'
port = 3306
username = 'xxxx'
password = 'xxxx'
conn = None

conn = connect_to_db(host_name, username, password, dbname)
curr = conn.cursor()

Connected to database


In [10]:
create_table(curr)

In [7]:
delete_table(curr)

In [13]:
new_vid_df = update_db(curr,df)
conn.commit()

In [14]:
new_vid_df

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,n8bLutlAfUc,SIDEMEN FAMILY FEUD 2,2022-02-06,6892324.0,386544.0,10559
1,xP8y54cbyk0,"SIDEMEN THAT FINDS $250,000 LAMBORGHINI WINS IT",2022-01-30,6352325.0,385047.0,13557
2,p1dfV9up_MY,SIDEMEN DRINK ONE COLOUR FOR 24 HOURS CHALLENGE,2022-01-23,8084022.0,328922.0,8370
3,Q9hy5ZxXaZA,"SIDEMEN HIDE AND SEEK IN $20,000,000 SPANISH M...",2022-01-16,6279800.0,323609.0,8247
4,PeCBE33DYYI,THE GREATEST SIDEMEN MOMENTS 2021,2022-01-09,4907846.0,308143.0,8587
...,...,...,...,...,...,...
197,-cE-1EDV2Xs,SIDEMEN PRO CLUBS IS BACK!,2017-01-08,4099356.0,157812.0,7360
198,jXKOWvsz1To,THE SIDEMEN AWARDS,2016-10-09,2895485.0,83500.0,6874
199,J_NniTO22_A,SIDEMEN FC VS YOUTUBE ALLSTARS CHARITY FOOTBAL...,2016-06-03,25328450.0,436002.0,36533
200,5j_fRfbscaE,SIDEMEN FACE PAINTING CHALLENGE,2016-05-28,4114828.0,135618.0,16430


In [15]:
new_vid_df = new_vid_df.dropna()
new_vid_df.isnull().sum()

video_id         0
video_title      0
upload_date      0
view_count       0
like_count       0
comment_count    0
dtype: int64

In [18]:
append_from_df_to_db(curr, new_vid_df)
conn.commit()

In [20]:
curr.execute("SELECT * FROM videos")
print(curr.fetchall())

[{'video_id': '_3b6LZ2xAVs', 'video_title': 'SIDEMEN SPELLING BEE', 'upload_date': datetime.date(2018, 9, 2), 'view_count': 28467766, 'like_count': 689252, 'comment_count': 24624}, {'video_id': '_unSKgaE47E', 'video_title': 'COUNTDOWN: SIDEMEN EDITION', 'upload_date': datetime.date(2019, 4, 22), 'view_count': 18983659, 'like_count': 420379, 'comment_count': 16056}, {'video_id': '-cE-1EDV2Xs', 'video_title': 'SIDEMEN PRO CLUBS IS BACK!', 'upload_date': datetime.date(2017, 1, 8), 'view_count': 4099356, 'like_count': 157812, 'comment_count': 7360}, {'video_id': '-SNXrJDkftY', 'video_title': 'SIDEMENS GOT TALENT', 'upload_date': datetime.date(2021, 3, 28), 'view_count': 13095333, 'like_count': 583134, 'comment_count': 36547}, {'video_id': '0_9fyL_vD4M', 'video_title': 'SIDEMEN WORLD CUP FOOTBALL CHALLENGES', 'upload_date': datetime.date(2018, 7, 15), 'view_count': 20205257, 'like_count': 292076, 'comment_count': 9769}, {'video_id': '09FaAy42iMI', 'video_title': 'SIDEMEN YOGA CHALLENGE', 'u

197